In [2]:
import os

In [7]:
localfilepath = "C:\\Users\\budde\\Desktop\\ds4300\\"
filepath = "practical_02_lon_kalden_desiree\\data\\"

In [ ]:
word_docs = {}
file_list = os.listdir(localfilepath + filepath)
for file in file_list:
    words = []
    with open(localfilepath + filepath + file, mode="r") as infile:
        for line in infile.readlines():
            words = words + line.strip().split()
        word_docs[file] = words

In [14]:
word_docs

{'02 - Foundations.txt': ['DS',
  '4300',
  'Large',
  'Scale',
  'Information',
  'Storage',
  'and',
  'Retrieval',
  'Foundations',
  'Mark',
  'Fontenot,',
  'PhD',
  'Northeastern',
  'University',
  'Searching',
  'Searching',
  'is',
  'the',
  'most',
  'common',
  'operation',
  'performed',
  'by',
  'a',
  'database',
  'system',
  'In',
  'SQL,',
  'the',
  'SELECT',
  'statement',
  'is',
  'arguably',
  'the',
  'most',
  'versatile',
  '/',
  'complex.',
  'Baseline',
  'for',
  'efficiency',
  'is',
  'Linear',
  'Search',
  'Start',
  'at',
  'the',
  'beginning',
  'of',
  'a',
  'list',
  'and',
  'proceed',
  'element',
  'by',
  'element',
  'until:',
  'You',
  'find',
  'what',
  'you’re',
  'looking',
  'for',
  'You',
  'get',
  'to',
  'the',
  'last',
  'element',
  'and',
  'haven’t',
  'found',
  'it',
  '2',
  'Searching',
  'Record',
  '-',
  'A',
  'collection',
  'of',
  'values',
  'for',
  'attributes',
  'of',
  'a',
  'single',
  'entity',
  'instan